In [4]:
#installing dependencies
#FastAPI-send request and response
#unicorn-runs fastapi app
#langhcain-llm useage and easily with prompts
#langgrpah-multistep flow
#pydantic-defines input and output
#nest_asyncio-allow fasiapi to ru inside google colab without crashing
#pyngrok-to use outside the google colab
!pip install fastapi uvicorn langchain langgraph pydantic nest_asyncio pyngrok


In [9]:
!pip install langchain-openai
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.4 MB/s eta 0:00:00


In [10]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph


In [11]:
#agent memory
class TripState(TypedDict):
  destination: str
  days: int
  itinerary: str
  packing_list: str

In [35]:
llm = ChatOpenAI(model="gpt-4o-mini",api_key="")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

#creating itinerary(planning list)

In [14]:
def create_itinerary(state: TripState) -> TripState:
    prompt = f"Create a {state['days']}-day travel itinerary for a trip to {state['destination']}." #taking the question
    response = llm.invoke(prompt) # LLm is repponding for the question
    state["itinerary"] = response.content
    return state #returns the response


#Packing list needed

In [15]:
def suggest_packing_list(state: TripState) -> TripState:
    prompt = f"Based on this itinerary for {state['destination']}, suggest a packing list:\n{state['itinerary']}"
    response = llm.invoke(prompt)
    state["packing_list"] = response.content
    return state


#workflow

In [17]:
graph= StateGraph(TripState) #what is the state of the grpah

In [18]:
graph.add_node("create_itinerary",create_itinerary)

In [19]:
graph.add_node("suggest_packing_list",suggest_packing_list)

In [24]:
graph.set_entry_point("create_itinerary")  #starting point

In [21]:
graph.add_edge("create_itinerary", "suggest_packing_list")


In [22]:
graph.set_finish_point("suggest_packing_list")


In [23]:
app_graph = graph.compile()


#Building fast_API

In [25]:
from fastapi import FastAPI

In [27]:
from pydantic import BaseModel

In [28]:
app=FastAPI()

In [29]:
class TripRequest(BaseModel):
  destination:str
  days:int

In [30]:
@app.post("/plan-trip")
async def plan_trip(request:TripRequest):
  memory={"destination":request.destination,"days":request.days,"itinerary":"","packing_list":""}
  Result=app_graph.invoke(memory)
  return Result

#running FAsIAPI in colab

In [32]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok

In [34]:
nest_asyncio.apply()
ngrok.set_auth_token("32BtS6gXH8BPj4TnrINtkj4GFCx_RXgBchBSq8Y98b3LJLPj")
ngrok_tunnel=ngrok.connect(8000)
print("Public URL:",ngrok_tunnel.public_url)

uvicorn.run(app,host="0.0.0.0",port=8000)

INFO:     Started server process [138]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://df0a7d3e4d8f.ngrok-free.app
INFO:     157.130.87.238:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     157.130.87.238:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     157.130.87.238:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     157.130.87.238:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     157.130.87.238:0 - "POST /plan-trip HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [138]
